In [1]:
import os
import numpy as np
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from numpy import loadtxt
import xgboost 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def categorize(array):
    new_array=np.zeros_like(array)
    for i in range(0,array.shape[0]): 
        k=array[i]
        #print(k)
        if k>0.45:
            new_array[i,:]=2
        elif 0.3<k<0.45 :
            new_array[i,:]=1
        else: 
            new_array[i,:]=0
            
    return new_array

train_dataset=pd.read_csv('/home/navodini/Documents/NUS/Brats19/Brats_valid/radiomics_normalized.csv', header=0)
test_dataset=pd.read_csv('/home/navodini/Documents/NUS/Brats19/Brats_valid/radiomics_valid_normalized.csv', header=0)
categorical_features = ['f1_nec','f2_nec','f3_nec','f4_nec','f5_nec','f1_tc','f2_tc','f3_tc','f4_tc','f5_tc','FirstAxis1_nec','FirstAxis2_nec','FirstAxis3_nec','SecondAxis1_nec','SecondAxis2_nec','SecondAxis3_nec','ThirdAxis1_nec','ThirdAxis2_nec','ThirdAxis3_nec','EigenValues1_nec','EigenValues2_nec','EigenValues3_nec','FirstAxisLength_nec','SecondAxisLength_nec','ThirdAxisLength_nec','Centroid1_nec','Centroid2_nec','Centroid3_nec','MeridionalEccentricity_nec','EquatorialEccentricity_nec','FirstAxis1_tc','FirstAxis2_tc','FirstAxis3_tc','SecondAxis1_tc','SecondAxis2_tc','SecondAxis3_tc','ThirdAxis1_tc','ThirdAxis2_tc','ThirdAxis3_tc','EigenValues1_tc','EigenValues2_tc','EigenValues3_tc','FirstAxisLength_tc','SecondAxisLength_tc','ThirdAxisLength_tc','Centroid1_tc','Centroid2_tc','Centroid3_tc','MeridionalEccentricity_tc','EquatorialEccentricity_tc','FirstAxis1_wt','FirstAxis2_wt','FirstAxis3_wt','SecondAxis1_wt','SecondAxis2_wt','SecondAxis3_wt','ThirdAxis1_wt','ThirdAxis2_wt','ThirdAxis3_wt','EigenValues1_wt','EigenValues2_wt','EigenValues3_wt','FirstAxisLength_wt','SecondAxisLength_wt','ThirdAxisLength_wt','Centroid1_wt','Centroid2_wt','Centroid3_wt','MeridionalEccentricity_wt','EquatorialEccentricity_wt','kurtosis_necrosis','entropy_necrosis','histogram_necrosis','entropy_enhancement','histogram_enhancement','Age']
every_column_except_y= [col for col in train_dataset.columns if col not in ['Patient_ID','class','Survival','Survival_indays']]
every_column_except_ID_test= [col for col in test_dataset.columns if col not in ['Patient_ID']]
train_dataset.describe()

,f1_nec,f2_nec,f3_nec,f4_nec,f5_nec,f1_tc,f2_tc,f3_tc,f4_tc,f5_tc,...,EquatorialEccentricity_wt,kurtosis_necrosis,entropy_necrosis,histogram_necrosis,entropy_enhancement,histogram_enhancement,Age,Survival,class,Survival_indays
count,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,...,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000
mean,0.130636,0.147792,0.197474,0.249171,0.318888,0.276645,0.288784,0.303232,0.324787,0.388676,...,0.726635,0.180425,0.240941,0.345457,0.166235,0.241157,0.628000,0.438542,0.965517,438.541872
std,0.149794,0.147151,0.165520,0.184758,0.197406,0.228655,0.230053,0.227963,0.226117,0.225342,...,0.187967,0.141165,0.177968,0.226505,0.161997,0.174736,0.179287,0.359673,0.863904,359.673342
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005000,0.000000,5.000000
25%,0.034679,0.046921,0.077713,0.106436,0.164557,0.094097,0.102871,0.117611,0.142683,0.209877,...,0.628872,0.087602,0.105442,0.169876,0.055169,0.111279,0.520626,0.183500,0.000000,183.500000
50%,0.090946,0.106302,0.158777,0.222772,0.316456,0.233254,0.246853,0.271504,0.295122,0.358025,...,0.767172,0.146910,0.206348,0.318184,0.129641,0.215558,0.636183,0.368000,1.000000,368.000000
75%,0.174478,0.202308,0.280268,0.355198,0.436709,0.371924,0.385878,0.408683,0.424390,0.512346,...,0.870433,0.224796,0.342244,0.474347,0.228772,0.342297,0.744838,0.548000,2.000000,548.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.767000,2.000000,1767.000000


In [2]:
#best_14 = ['EquatorialEccentricity_wt'

X = train_dataset[every_column_except_y]
Y = train_dataset['Survival'].values.reshape(-1,1)

print(X.shape)
print(Y.shape)

X_test = X.values[180:,:]
y_test = Y[180:,:]
#X_train = np.append(X.values[:160,:],X.values[180:,:],axis=0)
#y_train = np.append(Y[:160,:],Y[180:,:],axis=0)
X_train = X.values[:180,:]
y_train = Y[:180,:]
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
# fit model on training data
model = xgboost.XGBRegressor(colsample_bytree=0.8,
                 gamma=0.3,                 
                 learning_rate=0.1,
                 max_depth=5,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.01,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
#print(y_pred.reshape(21,1))
#print(y_test)
predictions = categorize(y_pred.reshape(23,1))
#np.savetxt('/home/navodini/Documents/NUS/Brats19/predictions_train1.csv',y_pred*1000)
y_test_class = categorize(y_test)
# evaluate predictions
accuracy = accuracy_score(y_test_class, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
np.savetxt('/home/navodini/Documents/NUS/Brats19/predictions_train_180_203.csv',y_pred*1000)

(203, 76)
(203, 1)
Accuracy: 30.43%
